In [1]:
%matplotlib inline

## Datasets Preparation

- Create Balanced Data set from the good quality images of the 1.25x 
- This dataset will have 200 images for each class.
- the remain images will be moved to the Val and Test datasets.
- The split between the val - test will be 50%-50%.
- I want to measure the performance of balanced dataset compared to the original which received good score

In [1]:
# import YOLO model
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import os

## Comparison between Different Yolo Models
Currently only YOLO v8 and v11 support image classification
- YOLO v8
- YOLO v11


In [8]:
import os

yolo_checkpoints_folder = "/home/etaylor/code_projects/thesis/checkpoints/trichome_image_classification/yolo"

yolo_classification_checkpoints = {
    "YOLOv8": {
        "Nano": os.path.join(yolo_checkpoints_folder, "yolov8n-cls.pt"),
        "Small": os.path.join(yolo_checkpoints_folder, "yolov8s-cls.pt"),
        "Medium": os.path.join(yolo_checkpoints_folder, "yolov8m-cls.pt"),
        "Large": os.path.join(yolo_checkpoints_folder, "yolov8l-cls.pt"),
        "XLarge": os.path.join(yolo_checkpoints_folder, "yolov8x-cls.pt"),
    },
        "YOLOv11": {
        "Nano": os.path.join(yolo_checkpoints_folder, "yolo11n-cls.pt"),
        "Small": os.path.join(yolo_checkpoints_folder, "yolo11s-cls.pt"),
        "Medium": os.path.join(yolo_checkpoints_folder, "yolo11m-cls.pt"),
        "Large": os.path.join(yolo_checkpoints_folder, "yolo11l-cls.pt"),
        "XLarge": os.path.join(yolo_checkpoints_folder, "yolo11x-cls.pt"),
    },
}

Functions to fine tune each model and then evaluate it

In [9]:
def train_model(data_path, model_name, model_size, epochs=10, imgsz=128):
    # Load the model checkpoint
    checkpoint = yolo_classification_checkpoints[model_name][model_size]
    model = YOLO(checkpoint)
    
    # Train the model
    model.train(data=data_path, epochs=epochs, imgsz=imgsz)
    
    return model

def eval_model(model, test_data_path):
    # Get class names from the test dataset directory
    class_names = sorted(os.listdir(test_data_path))
    class_to_idx = {class_name: idx for idx, class_name in enumerate(class_names)}

    # Initialize lists for true labels (y_test) and predicted labels (y_pred)
    y_test = []
    y_pred = []

    # Initialize a list to store image data for plotting
    images_for_plotting = []

    # Process each class in the test directory
    for class_name in class_names:
        class_dir = os.path.join(test_data_path, class_name)
        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_name)
            
            # Load the image and run the model prediction
            img = cv2.imread(img_path)
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB for matplotlib
            results = model(img)
            
            # Get the predicted class index
            predicted_idx = int(results[0].probs.data.argmax())
            
            # Append true and predicted labels
            y_test.append(class_to_idx[class_name])
            y_pred.append(predicted_idx)
            
            # Store the image, true label, and predicted label for plotting
            images_for_plotting.append((img_rgb, class_name, class_names[predicted_idx]))

    # Evaluate using classification metrics
    classification_report_result = classification_report(y_test, y_pred, target_names=class_names)
    accuracy = accuracy_score(y_test, y_pred)
    confusion_matrix_result = confusion_matrix(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    return classification_report_result, accuracy, confusion_matrix_result, precision, recall, f1, images_for_plotting
    
def plot_results(images_for_plotting):
    # Plot images with true and predicted labels
    plt.figure(figsize=(12, 12))
    num_images = len(images_for_plotting)  # Display all images (adjust as needed)
    rows = (num_images // 4) + 1  # Adjust rows based on the number of images

    for i in range(num_images):
        img, true_label, pred_label = images_for_plotting[i]
        plt.subplot(rows, 4, i + 1)  # Dynamic row setting
        plt.imshow(img)
        plt.title(f"True: {true_label}\nPred: {pred_label}", color="green" if true_label == pred_label else "red")
        plt.axis('off')

    plt.tight_layout()
    plt.show()

In [11]:
import os

# iterate over each dataset in the datasets folder
balanced_datasets = "/home/etaylor/code_projects/thesis/classification_datasets/trichome_classification/trichome_dataset_125_good_quality/balanced_datasets"
data_paths = [os.path.join(balanced_datasets, dataset) for dataset in os.listdir(balanced_datasets)]

results = []  # List to store the evaluation results

for i, data_path in enumerate(data_paths):
    test_data_path = os.path.join(data_path, "test")
    # run the functions for each of the models
    for model_name in yolo_classification_checkpoints:
        for model_size in yolo_classification_checkpoints[model_name]:
            print(f"Training {model_name} {model_size} model...")
            model = train_model(data_path, model_name, model_size, epochs=10, imgsz=128)
            
            print(f"Evaluating {model_name} {model_size} model...")
            classification_report_result, accuracy, confusion_matrix_result, precision, recall, f1, images_for_plotting = eval_model(model, test_data_path)
            
            # create saving folder if does not exist
            if not os.path.exists(f"/home/etaylor/code_projects/thesis/checkpoints/trichome_image_classification/yolo/{model_name}"):
                os.makedirs(f"/home/etaylor/code_projects/thesis/checkpoints/trichome_image_classification/yolo/{model_name}")
            
            # save model
            model.save(f"/home/etaylor/code_projects/thesis/checkpoints/trichome_image_classification/yolo/{model_name}/{model_size}_dataset_{i}.pt")
            
            # Append the evaluation results to the list
            results.append({
                "Model_Name": model_name,
                "Model_Size": model_size,
                "Classification Report": classification_report_result,
                "Accuracy": accuracy,
                "Average Precision": precision,
                "Average Recall": recall,
                "Average f1": f1,
                "Confusion Matrix": confusion_matrix_result,
                "Images for Plotting": images_for_plotting
            })

Training YOLOv8 Nano model...
Ultralytics 8.3.49 🚀 Python-3.9.19 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce GTX 1080 Ti, 11165MiB)
engine/trainer: task=classify, mode=train, model=/home/etaylor/code_projects/thesis/checkpoints/trichome_image_classification/yolo/yolov8n-cls.pt, data=/home/etaylor/code_projects/thesis/classification_datasets/trichome_classification/trichome_dataset_125_good_quality/balanced_datasets/train_set_1, epochs=10, time=None, patience=100, batch=16, imgsz=128, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train180, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, 

train: Scanning /sise/home/etaylor/code_projects/thesis/classification_datasets/trichome_classification/trichome_dataset_125_good_quality/balanced_datasets/train_set_1/train... 600 images, 0 corrupt: 100%|██████████| 600/600 [00:00<?, ?it/s]
/home/etaylor/.conda/envs/detectron_fastai/lib/python3.9/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 6, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
val: Scanning /sise/home/etaylor/code_projects/thesis/classification_datasets/trichome_classification/trichome_dataset_125_good_quality/balanced_datasets/train_set_1/val... 264 images, 0 corrupt: 100%|██████████| 264/264 [00:00<?, ?it/s]


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 26 weight(decay=0.0), 27 weight(decay=0.0005), 27 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 128 train, 128 val
Using 8 dataloader workers
Logging results to /home/etaylor/runs/classify/train180
Starting training for 10 epochs...


/home/etaylor/.conda/envs/detectron_fastai/lib/python3.9/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 6, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(



      Epoch    GPU_mem       loss  Instances       Size


...

      10/10      1.64G     0.3153          8        128: 100%|██████████| 38/38 [00:02<00:00, 12.71it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 9/9 [00:00<00:00, 34.16it/s]

                   all      0.913          1



10 epochs completed in 0.041 hours.
Optimizer stripped from /home/etaylor/runs/classify/train271/weights/last.pt, 57.0MB
Optimizer stripped from /home/etaylor/runs/classify/train271/weights/best.pt, 57.0MB

Validating /home/etaylor/runs/classify/train271/weights/best.pt...
Ultralytics 8.3.49 🚀 Python-3.9.19 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce GTX 1080 Ti, 11165MiB)
YOLO11x-cls summary (fused): 227 layers, 28,336,259 parameters, 0 gradients, 110.3 GFLOPs
train: /sise/home/etaylor/code_projects/thesis/classification_datasets/trichome_classification/trichome_dataset_125_good_quality/balanced_datasets/train_set_5/train... found 600 images in 3 classes ✅ 
val: /sise/home/etaylor/code_projects/thesis/classification_datasets/trichome_classification/trichome_dataset_125_good_quality/balanced_datasets/train_set_5/val... found 264 images in 3 classes ✅ 
test: /sise/home/etaylor/code_projects/thesis/classification_datasets/trichome_classification/trichome_dataset_125_good_quality/balanced_d

               classes   top1_acc   top5_acc: 100%|██████████| 9/9 [00:00<00:00, 17.95it/s]


                   all      0.939          1
Speed: 0.0ms preprocess, 1.7ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to /home/etaylor/runs/classify/train271
Evaluating YOLOv11 XLarge model...

0: 128x128 amber 0.77, clear 0.20, cloudy 0.04, 11.5ms
Speed: 3.7ms preprocess, 11.5ms inference, 0.1ms postprocess per image at shape (1, 3, 128, 128)

0: 128x128 amber 0.98, cloudy 0.01, clear 0.00, 14.2ms
Speed: 2.3ms preprocess, 14.2ms inference, 0.1ms postprocess per image at shape (1, 3, 128, 128)

0: 128x128 amber 1.00, cloudy 0.00, clear 0.00, 12.9ms
Speed: 1.9ms preprocess, 12.9ms inference, 0.0ms postprocess per image at shape (1, 3, 128, 128)

0: 128x128 amber 0.50, cloudy 0.46, clear 0.05, 10.1ms
Speed: 2.0ms preprocess, 10.1ms inference, 0.0ms postprocess per image at shape (1, 3, 128, 128)

0: 128x128 amber 1.00, cloudy 0.00, clear 0.00, 12.6ms
Speed: 2.0ms preprocess, 12.6ms inference, 0.1ms postprocess per image at shape (1, 3, 128, 128)

0: 128x128 amber 0

In [12]:
import pandas as pd

results_df = pd.DataFrame(results)
results_df

,Model_Name,Model_Size,Classification Report,Accuracy,Average Precision,Average Recall,Average f1,Confusion Matrix,Images for Plotting
0,YOLOv8,Nano,precision recall f1-score ...,0.939623,0.941423,0.939623,0.940375,"[[86, 0, 3], [0, 21, 5], [1, 7, 142]]",[([[[125 141 114]\n [122 138 111]\n [123 139 1...
1,YOLOv8,Small,precision recall f1-score ...,0.898113,0.924077,0.898113,0.905037,"[[85, 1, 3], [0, 24, 2], [3, 18, 129]]",[([[[125 141 114]\n [122 138 111]\n [123 139 1...
2,YOLOv8,Medium,precision recall f1-score ...,0.932075,0.933631,0.932075,0.932732,"[[85, 1, 3], [0, 21, 5], [3, 6, 141]]",[([[[125 141 114]\n [122 138 111]\n [123 139 1...
3,YOLOv8,Large,precision recall f1-score ...,0.935849,0.937194,0.935849,0.935844,"[[89, 0, 0], [0, 21, 5], [6, 6, 138]]",[([[[125 141 114]\n [122 138 111]\n [123 139 1...
4,YOLOv8,XLarge,precision recall f1-score ...,0.947170,0.949643,0.947170,0.947857,"[[89, 0, 0], [0, 22, 4], [3, 7, 140]]",[([[[125 141 114]\n [122 138 111]\n [123 139 1...
5,YOLOv11,Nano,precision recall f1-score ...,0.928302,0.929781,0.928302,0.928913,"[[85, 1, 3], [0, 21, 5], [4, 6, 140]]",[([[[125 141 114]\n [122 138 111]\n [123 139 1...
6,YOLOv11,Small,precision recall f1-score ...,0.909434,0.932070,0.909434,0.914874,"[[86, 0, 3], [0, 25, 1], [3, 17, 130]]",[([[[125 141 114]\n [122 138 111]\n [123 139 1...
7,YOLOv11,Medium,precision recall f1-score ...,0.928302,0.929203,0.928302,0.928418,"[[87, 0, 2], [0, 21, 5], [6, 6, 138]]",[([[[125 141 114]\n [122 138 111]\n [123 139 1...
8,YOLOv11,Large,precision recall f1-score ...,0.947170,0.947688,0.947170,0.947366,"[[87, 0, 2], [0, 22, 4], [3, 5, 142]]",[([[[125 141 114]\n [122 138 111]\n [123 139 1...
9,YOLOv11,XLarge,precision recall f1-score ...,0.928302,0.926679,0.928302,0.927282,"[[88, 0, 1], [0, 18, 8], [4, 6, 140]]",[([[[125 141 114]\n [122 138 111]\n [123 139 1...


In [13]:
# sort the model results based on the f1 score
results_df = results_df.sort_values(by="Average f1", ascending=False)
results_df

,Model_Name,Model_Size,Classification Report,Accuracy,Average Precision,Average Recall,Average f1,Confusion Matrix,Images for Plotting
22,YOLOv8,Medium,precision recall f1-score ...,0.966038,0.968904,0.966038,0.966972,"[[87, 1, 1], [0, 24, 2], [0, 5, 145]]",[([[[62 70 31]\n [56 64 25]\n [56 64 25]\n [56...
37,YOLOv11,Medium,precision recall f1-score ...,0.962264,0.962801,0.962264,0.962468,"[[89, 0, 0], [0, 22, 4], [1, 5, 144]]",[([[[160 159 102]\n [162 161 104]\n [160 157 1...
34,YOLOv8,XLarge,precision recall f1-score ...,0.950943,0.950515,0.950943,0.950692,"[[87, 0, 2], [0, 21, 5], [2, 4, 144]]",[([[[160 159 102]\n [162 161 104]\n [160 157 1...
47,YOLOv11,Medium,precision recall f1-score ...,0.947170,0.956966,0.947170,0.949465,"[[85, 1, 3], [0, 26, 0], [1, 9, 140]]",[([[[111 132 31]\n [116 133 37]\n [116 133 ...
45,YOLOv11,Nano,precision recall f1-score ...,0.947170,0.950867,0.947170,0.948326,"[[84, 1, 4], [0, 23, 3], [0, 6, 144]]",[([[[111 132 31]\n [116 133 37]\n [116 133 ...
4,YOLOv8,XLarge,precision recall f1-score ...,0.947170,0.949643,0.947170,0.947857,"[[89, 0, 0], [0, 22, 4], [3, 7, 140]]",[([[[125 141 114]\n [122 138 111]\n [123 139 1...
8,YOLOv11,Large,precision recall f1-score ...,0.947170,0.947688,0.947170,0.947366,"[[87, 0, 2], [0, 22, 4], [3, 5, 142]]",[([[[125 141 114]\n [122 138 111]\n [123 139 1...
40,YOLOv8,Nano,precision recall f1-score ...,0.943396,0.947717,0.943396,0.944845,"[[86, 0, 3], [0, 23, 3], [1, 8, 141]]",[([[[111 132 31]\n [116 133 37]\n [116 133 ...
38,YOLOv11,Large,precision recall f1-score ...,0.943396,0.946904,0.943396,0.944657,"[[87, 1, 1], [0, 22, 4], [2, 7, 141]]",[([[[160 159 102]\n [162 161 104]\n [160 157 1...
49,YOLOv11,XLarge,precision recall f1-score ...,0.943396,0.945744,0.943396,0.944281,"[[86, 2, 1], [0, 22, 4], [3, 5, 142]]",[([[[111 132 31]\n [116 133 37]\n [116 133 ...


In [14]:
# create an average result dataframe for each of the models
average_results = []

for model_name in yolo_classification_checkpoints:
    for model_size in yolo_classification_checkpoints[model_name]:
        model_results = results_df[(results_df["Model_Name"] == model_name) & (results_df["Model_Size"] == model_size)]
        
        # Calculate the average accuracy
        avg_accuracy = model_results["Accuracy"].mean()
        avg_precision = model_results["Average Precision"].mean()
        avg_recall = model_results["Average Recall"].mean()
        avg_f1 = model_results["Average f1"].mean()
        
        # Calculate the average confusion matrix
        avg_confusion_matrix = model_results["Confusion Matrix"].mean(axis=0)
        
        # Append the average results to the list
        average_results.append({
            "Model Name": model_name,
            "Model Size": model_size,
            "Average Accuracy": avg_accuracy,
            "Average Precision": avg_precision,
            "Average Recall": avg_recall,
            "Average f1": avg_f1,
            "Average Confusion Matrix": avg_confusion_matrix
        })

In [15]:
average_results_df = pd.DataFrame(average_results)

# sort the average results based on the f1 score
average_results_df = average_results_df.sort_values(by="Average f1", ascending=False)

average_results_df


,Model Name,Model Size,Average Accuracy,Average Precision,Average Recall,Average f1,Average Confusion Matrix
7,YOLOv11,Medium,0.939623,0.942376,0.939623,0.940035,"[[87.0, 0.2, 1.8], [0.0, 22.2, 3.8], [3.8, 6.4..."
2,YOLOv8,Medium,0.935849,0.938424,0.935849,0.936804,"[[85.6, 0.4, 3.0], [0.0, 21.6, 4.4], [2.2, 7.0..."
8,YOLOv11,Large,0.935849,0.938533,0.935849,0.936804,"[[85.0, 0.6, 3.4], [0.0, 22.0, 4.0], [2.4, 6.6..."
9,YOLOv11,XLarge,0.931321,0.932426,0.931321,0.931424,"[[86.4, 0.8, 1.8], [0.0, 20.2, 5.8], [3.6, 6.2..."
5,YOLOv11,Nano,0.926038,0.933875,0.926038,0.928361,"[[84.2, 0.8, 4.0], [0.0, 22.6, 3.4], [2.0, 9.4..."
4,YOLOv8,XLarge,0.924528,0.936192,0.924528,0.927342,"[[86.6, 0.0, 2.4], [0.0, 22.0, 4.0], [2.6, 11...."
3,YOLOv8,Large,0.923774,0.934698,0.923774,0.926850,"[[86.6, 0.2, 2.2], [0.0, 22.4, 3.6], [2.6, 11...."
0,YOLOv8,Nano,0.923019,0.934474,0.923019,0.926043,"[[83.6, 0.8, 4.6], [0.0, 23.4, 2.6], [1.6, 10...."
1,YOLOv8,Small,0.920755,0.929032,0.920755,0.922969,"[[84.6, 0.8, 3.6], [0.0, 22.2, 3.8], [3.4, 9.4..."
6,YOLOv11,Small,0.920755,0.929185,0.920755,0.922652,"[[83.8, 0.4, 4.8], [0.0, 22.6, 3.4], [3.0, 9.4..."


In [33]:
# # save the results to a file
# average_results_df.to_csv("yolo_classification_avg_results_for_5_datasets.csv", index=False)
# results_df.to_csv("yolo_classification_results_for_5_datasets.csv", index=False)

In [36]:
print(average_results_df.columns)

Index(['Model Name', 'Model Size', 'Average Accuracy', 'Average Precision',
       'Average Recall', 'Average f1', 'Average Confusion Matrix'],
      dtype='object')


In [18]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Convert the average results to a DataFrame
average_results_df = pd.DataFrame(average_results)

# Plot the key metrics (accuracy, precision, recall, F1) across different models and sizes
metrics = ["Average Accuracy", "Average Precision", "Average Recall", "Average f1"]

plt.figure(figsize=(16, 12))

for i, metric in enumerate(metrics, 1):
    plt.subplot(2, 2, i)
    sns.barplot(data=average_results_df, x="Model Size", y=metric, hue="Model Name")
    plt.title(f"Comparison of {metric} Across Models and Sizes")
    plt.ylabel(metric)
    plt.xlabel("Model Size")
    plt.legend(title="Model Name")

plt.tight_layout()
plt.show()


<Figure size 1600x1200 with 4 Axes>

In [19]:
import matplotlib.pyplot as plt
import seaborn as sns

# Define the number of rows and columns for the grid
num_matrices = len(average_results_df)
ncols = 3  # Number of columns in the grid
nrows = (num_matrices // ncols) + (num_matrices % ncols > 0)  # Calculate rows needed

# Set up the figure for the grid of confusion matrices
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(ncols * 5, nrows * 5))

# Flatten axes to easily iterate over them
axes = axes.flatten()

# Plot each confusion matrix in the grid
for index, (ax, row) in enumerate(zip(axes, average_results_df.iterrows())):
    row_data = row[1]
    sns.heatmap(row_data["Average Confusion Matrix"], annot=True, fmt=".2f", cmap="Blues", cbar=True, ax=ax)
    ax.set_title(f"{row_data['Model Name']} - {row_data['Model Size']}")
    ax.set_xlabel("Predicted Class")
    ax.set_ylabel("True Class")

# Remove any unused subplots
for i in range(num_matrices, len(axes)):
    fig.delaxes(axes[i])

plt.tight_layout()
plt.show()


<Figure size 1500x2000 with 20 Axes>

## Experiment Settings
- 200 Images for each class for training
- The rest of the images goes to val and test sets
- Val and test sets splits are 50%-50% (26 - clear, 150 - cloudy and 89 - amber )

## Experiment results
- Good Results
- Around 93% for all metrics (accuracy, precision, recall and f1)
- In conf matrix class 0 = amber, 1 = clear and 2 = cloudy
- Best model to predict amber class is yolov8 Large.
